# 🌱 Lekce 5 – Řešení krok za krokem (Binární stromy)

Vítám tě v řešeních ke **Lekci 5**! Tento notebook je připravený hlavně pro ty, kteří se chtějí v klidu naučit, jak fungují algoritmy nad binárními stromy. Všechno je rozebrané **pomalu, s obrázky v hlavě a komentovaným kódem**, aby bylo jasné, **proč** jednotlivé kroky děláme.

Co tě čeká:

- 💡 Vysvětlení základních pojmů (výška stromu, průchody)
- 🧩 Tři populární LeetCode úlohy se snadno čitelným kódem
- ✅ Testy, které si můžeš rovnou pustit a sledovat výsledky
- 📌 Doporučené kroky pro vlastní experimentování

> ? Nejprve se podívej na pomocné funkce – díky nim si můžeme stavět stromy z obyčejných seznamů a kontrolovat výsledky.


## 🛠️ Pomocné funkce pro testování

Abychom mohli snadno testovat řešení, definujeme společnou strukturu `TreeNode` a helper funkce pro sestavení stromu z pole (úroveň po úrovni) a pro tisk stromu.


In [ ]:
from collections import deque


class TreeNode(object):
    """Jednoduchý uzel binárního stromu – stejný jako na LeetCode."""

    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def __repr__(self):
        return f"TreeNode(val={self.val})"


def build_tree(values):
    """Postaví strom z úrovňového seznamu (None = chybějící uzel)."""
    if not values:
        return None

    root = TreeNode(values[0])
    queue = deque([root])
    idx = 1

    while queue and idx < len(values):
        node = queue.popleft()

        if idx < len(values) and values[idx] is not None:
            node.left = TreeNode(values[idx])
            queue.append(node.left)
        idx += 1

        if idx < len(values) and values[idx] is not None:
            node.right = TreeNode(values[idx])
            queue.append(node.right)
        idx += 1

    return root


def level_order(root):
    """Vrátí úrovňový průchod stromu jako seznam hodnot."""
    if not root:
        return []

    result = []
    queue = deque([root])

    while queue:
        node = queue.popleft()
        if node:
            result.append(node.val)
            queue.append(node.left)
            queue.append(node.right)
        else:
            result.append(None)

    while result and result[-1] is None:
        result.pop()
    return result


def pretty_tree(root):
    """Jemná textová vizualizace stromu po úrovních."""
    if not root:
        return "<empty>"

    lines = []
    queue = deque([(root, 0)])
    current_level = 0
    level_nodes = []

    while queue:
        node, level = queue.popleft()
        if level != current_level:
            lines.append(" ".join(level_nodes))
            level_nodes = []
            current_level = level

        if node:
            level_nodes.append(str(node.val))
            queue.append((node.left, level + 1))
            queue.append((node.right, level + 1))
        else:
            level_nodes.append("·")

    if level_nodes:
        lines.append(" ".join(level_nodes))

    return "\n".join(lines)

## 🌳 Úloha 1: Diameter of Binary Tree (LeetCode 543)

### Co vlastně hledáme?
- Hledáme **nejdelší cestu** mezi dvěma uzly ve stromu.
- Počítáme **hrany**, nikoli uzly.
- Cesta může jít přes kořen, ale nemusí – může být celá v levém nebo pravém podstromu.

### Jak na to v klidu:
1. Pomocí **rekurze** si u každého uzlu zjistíme výšku levého a pravého podstromu.
2. Vždy zkusíme, jestli součet těchto výšek neudělá delší cestu, než jsme zatím našli.
3. Nahoru vracíme jen „nejvyšší větev“, protože rodič potřebuje vědět, jak vysoko jeho děti sahají.

### Proč to funguje?
- Nejdelší cesta musí probíhat přes nějaký uzel X.
- V uzlu X jde cesta z nejhlubšího listu vlevo do nejhlubšího listu vpravo → právě proto sčítáme dvě výšky.

### Použité pojmy
- **Výška stromu** = kolik hran je z uzlu do nejvzdálenějšího listu.
- **Průměr (diameter)** = největší součet výšek levé a pravé strany (počítáme hrany).

> Zní to složitě? Neboj – v kódu je to pár řádků s komentáři. 😊

In [ ]:
class SolutionDiameter(object):
    """Jednoduché rekurzivní řešení pro LeetCode 543."""

    def diameterOfBinaryTree(self, root):
        # Tohle bude naše globální maximum – nejdelší nalezená cesta v počtu hran
        self.longest_path = 0

        def height(node):
            """Vrátí výšku podstromu (kolik hran je od uzlu po nejdelší list)."""
            if not node:
                return 0  # prázdný podstrom má výšku 0

            # Zjistíme výšky levé a pravé větve
            left = height(node.left)
            right = height(node.right)

            # Zkusíme, jestli cesta přes tento uzel není nejdelší, co jsme zatím viděli
            path_length = left + right
            if path_length > self.longest_path:
                self.longest_path = path_length

            # Nahoru vracíme výšku – bereme vyšší větev + hranu k rodiči
            return max(left, right) + 1

        height(root)
        return self.longest_path


class SolutionDiameterIterative(object):
    """Bonus: iterativní varianta se zásobníkem (pouze pro srovnání)."""

    def diameterOfBinaryTree(self, root):
        if not root:
            return 0

        longest = 0
        stack = [(root, False)]  # False = ještě jsme uzel nezpracovali
        heights = {}

        while stack:
            node, visited = stack.pop()
            if not node:
                continue

            if visited:
                left = heights.get(node.left, 0)
                right = heights.get(node.right, 0)
                longest = max(longest, left + right)
                heights[node] = max(left, right) + 1
            else:
                stack.append((node, True))
                stack.append((node.left, False))
                stack.append((node.right, False))

        return longest


def run_diameter_tests():
    print("🌳 Testy pro Diameter of Binary Tree")
    print("-" * 40)

    examples = [
        ("Ukázka z LeetCode", [1, 2, 3, 4, 5], 3),
        ("Malý strom se dvěma uzly", [1, 2], 1),
        ("Jeden uzel", [1], 0),
        ("Větev (všechno vpravo)", [1, None, 2, None, 3, None, 4], 3),
        ("Větev (všechno vlevo)", [1, 2, None, 3, None, 4], 4),
        ("Prázdný strom", [], 0),
    ]

    for name, values, expected in examples:
        root = build_tree(values)
        got = SolutionDiameter().diameterOfBinaryTree(root)
        status = "✅" if got == expected else "❌"
        print(f"{status} {name}")
        print(f"   Vstup: {values}")
        print(f"   Očekávaný výsledek: {expected}")
        print(f"   Výsledek funkce:     {got}\n")

    print("✨ Tip: spusť si i Bonus řešení a porovnej výsledky.")


run_diameter_tests()

In [ ]:
# Definition for a binary tree node.
# class TreeNode(object):
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
class Solution(object):
    def buildTree(self, preorder, inorder):
        """
        :type preorder: List[int]
        :type inorder: List[int]
        :rtype: Optional[TreeNode]
        """
        

## 🌲 Úloha 2: Maximum Depth of Binary Tree (LeetCode 104)

### Co si představit
- Hloubka stromu je **nejdelší cesta od kořene k listu**, tentokrát počítáme **uzly**.
- Když strom nemá žádný uzel (je prázdný), hloubka je 0.

### Postup po malých krocích
1. Když narazíme na `None`, vracíme 0 – žádná hloubka.
2. Zjistíme hloubku levého i pravého podstromu.
3. Vezmeme větší číslo a přičteme 1 za aktuální uzel.

### Varianty řešení
- **Rekurzivně** (nejjednodušší na pochopení).
- **Pomocí BFS** (projde strom po úrovních a počítá, kolik úrovní jsme navštívili).

> Vyzkoušej si obě verze a sleduj, že vrací stejný výsledek. 😊

In [ ]:
class SolutionMaxDepth(object):
    """Rekurzivní řešení – nejjednodušší na pochopení."""

    def maxDepth(self, root):
        if not root:
            return 0
        left_depth = self.maxDepth(root.left)
        right_depth = self.maxDepth(root.right)
        return max(left_depth, right_depth) + 1


class SolutionMaxDepthBFS(object):
    """Bonus: iterativně pomocí fronty (úrovňový průchod)."""

    def maxDepth(self, root):
        if not root:
            return 0

        depth = 0
        queue = deque([root])

        while queue:
            depth += 1
            for _ in range(len(queue)):
                node = queue.popleft()
                if node.left:
                    queue.append(node.left)
                if node.right:
                    queue.append(node.right)
        return depth


def run_max_depth_tests():
    print("🌲 Testy pro Maximum Depth")
    print("-" * 40)

    examples = [
        ("Ukázka z LeetCode", [3, 9, 20, None, None, 15, 7], 3),
        ("Jeden uzel", [1], 1),
        ("Prázdný strom", [], 0),
        ("Větev vpravo", [1, None, 2, None, 3], 3),
        ("Větev vlevo", [1, 2, None, 3, None, 4], 4),
    ]

    for name, values, expected in examples:
        root = build_tree(values)
        recursive = SolutionMaxDepth().maxDepth(root)
        bfs = SolutionMaxDepthBFS().maxDepth(root)
        status = "✅" if recursive == expected == bfs else "❌"
        print(f"{status} {name}")
        print(f"   Vstup: {values}")
        print(f"   Rekurzivně: {recursive}")
        print(f"   BFS:        {bfs}")
        print(f"   Očekáváno:  {expected}\n")


run_max_depth_tests()

## 🏗️ Úloha 3: Construct Binary Tree from Preorder and Inorder Traversal (LeetCode 105)

### Představ si dvě mapy
- **Preorder**: nejdřív kořen, pak levý podstrom, pak pravý podstrom.
- **Inorder**: levý podstrom, kořen, pravý podstrom.

Z těchto dvou seznamů dokážeme strom složit jako puzzle:
1. Vezmeme **první číslo z preorder** → to je kořen.
2. V inorder najdeme, kde se kořen nachází → nalevo jsou uzly levého podstromu, napravo pravého.
3. Stejný postup zopakujeme rekurzivně pro obě poloviny.

### Dvě varianty
- **Rychlá**: využije slovník (hash map), aby rychle našla pozici v inorder.
- **Jednodušší na pochopení**: pracuje s řezy seznamu (slice) – je pomalejší, ale pěkně vidíš, co se děje.

> Doporučení: začni jednoduchou verzí se slicingem, pak přidej slovník pro lepší výkon.


In [ ]:
class SolutionBuildTree(object):
    """Rychlé řešení se slovníkem – ideální na LeetCode."""

    def buildTree(self, preorder, inorder):
        if not preorder or not inorder:
            return None

        index_map = {value: idx for idx, value in enumerate(inorder)}
        preorder_index = [0]  # zabalené v listu, aby se dalo měnit uvnitř funkce

        def helper(left, right):
            if left > right:
                return None

            root_value = preorder[preorder_index[0]]
            preorder_index[0] += 1

            root = TreeNode(root_value)
            mid = index_map[root_value]

            root.left = helper(left, mid - 1)
            root.right = helper(mid + 1, right)
            return root

        return helper(0, len(inorder) - 1)


class SolutionBuildTreeSlice(object):
    """Pomalejší, ale přehledná verze bez slovníku (pro pochopení)."""

    def buildTree(self, preorder, inorder):
        if not preorder:
            return None

        root_value = preorder[0]
        root = TreeNode(root_value)
        mid = inorder.index(root_value)

        root.left = self.buildTree(preorder[1 : mid + 1], inorder[:mid])
        root.right = self.buildTree(preorder[mid + 1 :], inorder[mid + 1 :])
        return root


def run_build_tree_tests():
    print("🏗️ Testy pro Construct Binary Tree")
    print("-" * 40)

    examples = [
        (
            "Ukázka z LeetCode",
            [3, 9, 20, 15, 7],
            [9, 3, 15, 20, 7],
            [3, 9, 20, None, None, 15, 7],
        ),
        ("Pouze jeden uzel", [-1], [-1], [-1]),
        (
            "Kompletní strom",
            [1, 2, 4, 5, 3],
            [4, 2, 5, 1, 3],
            [1, 2, 3, 4, 5],
        ),
    ]

    for name, preorder, inorder, expected in examples:
        fast_tree = SolutionBuildTree().buildTree(preorder[:], inorder[:])
        slow_tree = SolutionBuildTreeSlice().buildTree(preorder[:], inorder[:])
        fast_level = level_order(fast_tree)
        slow_level = level_order(slow_tree)
        status = "✅" if fast_level == expected == slow_level else "❌"

        print(f"{status} {name}")
        print(f"   Preorder: {preorder}")
        print(f"   Inorder:  {inorder}")
        print(f"   Rychlé řešení → {fast_level}")
        print(f"   Jednodušší řešení → {slow_level}")
        print(f"   Očekáváno → {expected}\n")


run_build_tree_tests()

---

## ? Shrnutí pro začátečníky

| Úloha | Hlavní myšlenka | Co si zapamatovat |
|-------|-----------------|-------------------|
| Diameter of Binary Tree | DFS + výška podstromů | Nejdelší cesta = levá výška + pravá výška |
| Maximum Depth | Rekurze / BFS | Hloubka prázdného stromu je 0, nezapomeň přičíst 1 za aktuální uzel |
| Construct Tree | Rozdělení podle inorder | V preorder je kořen vždy první |

### Co můžeš zkusit dál
1. Přidej si do stromu vizualizaci pomocí `pretty_tree(root)` a vytiskni si jednotlivé případy.
2. Změň hodnoty ve vstupních polích a sleduj, jak se mění výsledky.
3. Vyzkoušej další stromové úlohy: **110. Balanced Binary Tree**, **226. Invert Binary Tree**.

> Největší pokrok uděláš, když si kód budeš upravovat a hrát si s ním. Neboj se experimentovat! 🌟